# МУХА

> Бык с плугом на покой тащился по трудах \
А Муха у него сидела на рогах,\
И Муху же они дорогой повстречали.\
«Откуда ты, сестра?» — от этой был вопрос.\
А та, поднявши нос,\
В ответ ей говорит: «Откуда? — мы пахали!»\
От басни завсегда\
Нечаянно дойдешь до были.\
Случалось ли подчас вам слышать, господа:\
«Мы сбили! Мы решили!»\
*(И.И. Дмитриев)*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
df['totwrk2_10000'] = df['totwrk']**2/10000

In [ ]:
model = smf.ols(data = df , formula = 'sleep~I(totwrk/100)+totwrk2_10000 +  age + smsa + male + south').fit()
model_1 = smf.ols(data = df , formula = 'I(totwrk/100)~totwrk2_10000 +  age + smsa + male + south').fit()
model_2 = smf.ols(data = df , formula = 'totwrk2_10000~ I(totwrk/100)+  age + smsa + male + south').fit()
model_3 = smf.ols(data = df , formula = 'male~I(totwrk/100)+ I(totwrk/100)+  age + smsa  + south').fit()
model_1.params.round(3)

In [ ]:
model_2.params.round(3)

In [ ]:
model_3.params.round(3)

In [ ]:
model.params

In [ ]:
model.rsquared

In [ ]:
VIF_1 = 1/(1 - model_1.rsquared.round(3))
VIF_1.round(3)

In [ ]:
VIF_2 = 1/(1 - model_2.rsquared.round(3))
VIF_2.round(3)

In [ ]:
VIF_3 = 1/(1 - model_3.rsquared.round(3))
VIF_3.round(3)

## Рассмотрим первую модель в деталях

In [ ]:
X = df[['totwrk2_10000' , 'age',  'smsa', 'male', 'south']]
X

## Формула для вычисления коэффициентов регрессии известна
## $$\hat{\beta} = (X' X)^{-1}(X')Y$$

In [ ]:
np.linalg.inv(X.T@X).round(2)

In [ ]:
np.linalg.det(np.linalg.inv(X.T@X).round(5))

## <span style="color:red"> Вывод.</span>
## Из курса линейной алгебры известно, что определитель матрицы равен нулю, если её столбцы/строки являются линейно зависимыми. Равенство нулю определителя матрицы $(X' X)^{-1}$ указывает на то, что в матрице есть зависимые параметры. В конкретно нашем примере это параметры $totwrk^2/1000$ и $totwrk/100$.